### Import Packages

In [3]:
import os
import io
import tensorflow as tf
import numpy as np
from azure.storage.blob import BlobServiceClient, ContainerClient
from PIL import Image
import matplotlib.pyplot as plt
import ast

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model



### Connect to Azure

In [5]:
#set up storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=mlfinalexam5505462853;AccountKey=0c40lghglG5/GlNK9yujDQAgo38GKoS2I3DeC/g22hwAEIFANKpmC/TqOpRk4RCT1DbfNiHBFt72+AStB+PfUA==;EndpointSuffix=core.windows.net"
container_name = "datacomplete"

#create client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

### Load Image Paths and Labels

In [6]:
#get filepaths
import pandas as pd
df = pd.read_csv("/Users/jonathan/Library/Mobile Documents/com~apple~CloudDocs/Master/2_Semester/ML/Assignments/machine_learning/ML_Final_Project/labeled_train.csv")

# Assuming 'df' is your DataFrame
df = df.sample(n=3000, random_state=42)

filenames = df["filename"]
# convert each string in the DataFrame to a list
df['labels'] = df['labels'].apply(ast.literal_eval)

# convert each list in the DataFrame to a numpy array
df['labels'] = df['labels'].apply(np.array)

# store all the arrays in a list
labels = df['labels'].tolist()

### Train Test Split

In [7]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(df['filename'], df['labels'], test_size=0.2)

# First split: Splitting into 80% for the temporary training set and 20% for the test set
X_temp, X_test, y_temp, y_test = train_test_split(df['filename'], df['labels'], test_size=0.15, random_state=42)

# Second split: Splitting the temporary training set into 87.5% for training and 12.5% for validation
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42)  # 0.125 * 0.8 = 0.1


#convert train and test labels to arrays
y_train = y_train.apply(np.array).tolist()
y_test = y_test.apply(np.array).tolist()
y_val = y_val.apply(np.array).tolist()

#remove labels
y_train = [arr[[1,5,6]] for arr in y_train]
y_test = [arr[[1,5,6]] for arr in y_test]
y_val = [arr[[1,5,6]] for arr in y_val]

print("Training Set: ", len(X_train))
print("Test Set: ", len(X_test))
print("Validation Set", len(X_val))

Training Set:  700
Test Set:  150
Validation Set 150


### Set Variables

In [8]:
image_size=224
channels=3

batch_size = 224 # Big enough to measure an F1-score
autotune = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically
shuffle_buffer_size = 1024 # Shuffle the training data by a chunck of 1024 observations

### Functions to create data (input for models)

In [9]:
def load_image(path):
    """Load an image from Azure Blob Storage."""
    blob_client = container_client.get_blob_client(path)
    blob_data = blob_client.download_blob().readall()  # Directly read all bytes
    return io.BytesIO(blob_data)

def load_and_preprocess_image(path):
    """Loads an image, decodes it to grayscale, resizes, and normalizes it."""
    # Load image
    image_file = load_image(path.numpy().decode('utf-8'))
    # Decode the image to grayscale
    image_tensor = tf.io.decode_image(image_file.getvalue(), channels=channels)
    # Resize the image
    image_resized = tf.image.resize(image_tensor, [image_size, image_size])
    # Normalize the image data
    image_normalized = image_resized / 255.0
    return image_normalized


def process_tensor(path, label):
    """Function to load an image from blob storage, decode, resize, and normalize it."""
    image_normalized = tf.py_function(load_and_preprocess_image, [path], tf.float32)
    # Ensure the shape is set correctly for grayscale
    image_normalized.set_shape([image_size, image_size, channels])
    return image_normalized, label

def create_dataset(filenames, labels, is_training=True):
    """Creates a TensorFlow dataset from filenames and labels."""
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.map(process_tensor, num_parallel_calls=tf.data.AUTOTUNE)
    
    if is_training:
        dataset = dataset.cache()
        dataset = dataset.shuffle(buffer_size=1024)
        
    dataset = dataset.batch(256)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset


### Create the dataset

In [10]:
train_ds = create_dataset(X_train, y_train)
test_ds = create_dataset(X_test, y_test, False)
val_ds = create_dataset(X_val, y_val, False)

### Print the dataset

In [ ]:
for images, labels in train_ds.take(1):  # Here, take(1) takes the first batch
    print("Images:", images.numpy())  # Convert tensor to numpy array and print
    print("Labels:", labels.numpy())  # Convert tensor to numpy array and print


In [51]:
#check dataset shape
for f, l in train_ds.take(1):
    print("Shape of features array:", f.numpy().shape)
    print("Shape of labels array:", l.numpy().shape)

Shape of features array: (256, 224, 224, 3)
Shape of labels array: (256, 7)


2024-05-10 10:50:39.675819: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Plot one image from the dataset

In [ ]:
def plot_first_image_from_dataset(dataset, index):
    # Take one batch from the dataset
    for images, labels in dataset.take(index):
        # Assuming the image tensor is in the shape [batch_size, height, width, channels]
        # and you need the first image in the batch
        first_image = images[0]  # This is a tensor

        # Check if the image needs to be squeezed (in case it's a grayscale image with a single channel)
        if first_image.shape[-1] == 1:
            first_image = tf.squeeze(first_image, axis=-1)
        
        # Convert tensor to numpy for plotting
        first_image_np = first_image.numpy()

        # Plot the image
        plt.imshow(first_image_np, cmap='gray')
        plt.title(f'Label: {labels[0].numpy()}')
        plt.axis('off')
        plt.show()

# Example usage with your train_ds dataset
plot_first_image_from_dataset(train_ds,1)

### Self Trained VGG
Source: https://medium.com/@siddheshb008/vgg-net-architecture-explained-71179310050f

In [ ]:
_input = Input((224,224,1)) 

conv1  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(_input)
conv2  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(conv1)
pool1  = MaxPooling2D((2, 2))(conv2)

conv3  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(pool1)
conv4  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(conv3)
pool2  = MaxPooling2D((2, 2))(conv4)

conv5  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(pool2)
conv6  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv5)
conv7  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv6)
pool3  = MaxPooling2D((2, 2))(conv7)

conv8  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool3)
conv9  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv8)
conv10 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv9)
pool4  = MaxPooling2D((2, 2))(conv10)

conv11 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool4)
conv12 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv11)
conv13 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv12)
pool5  = MaxPooling2D((2, 2))(conv13)

flat   = Flatten()(pool5)
dense1 = Dense(4096, activation="relu")(flat)
dense2 = Dense(4096, activation="relu")(dense1)
output = Dense(8, activation="sigmoid")(dense2) #adapted number of outputs and outputfunction

vgg16_model  = Model(inputs=_input, outputs=output)

LR = 1e-5 #why?
EPOCHS = 10 #why?


#compile the model
vgg16_model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=LR),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

##provide a model summary
vgg16_model.summary()

#fit the model
vgg16_model.fit(
    train_ds,
    validation_split = 0.2,
    epochs=EPOCHS
)

In [ ]:
#get prediction
predictions = vgg16_model.predict(train_ds)
predictions


### Plot model

In [ ]:
#from keras.utils.vis_utils import plot_model
from keras.utils import plot_model
plot_model(model, 
           to_file='vgg.png',
           show_shapes=True,
           show_dtype=True,
           show_layer_names=True,
           show_layer_activations=True,
           show_trainable=False,)

### Pre-trained

MobileNetV2: https://github.com/ashrefm/multi-label-soft-f1/blob/master/Multi-Label%20Image%20Classification%20in%20TensorFlow%202.0.ipynb

In [ ]:
from tensorflow.keras import layers
import tensorflow_hub as hub


IMG_SIZE = 224
CHANNELS = 3
feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(IMG_SIZE,IMG_SIZE,CHANNELS))

feature_extractor_layer.trainable = False

model_mnv2 = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(1024, activation='relu', name='hidden_layer'),
    layers.Dense(7, activation='sigmoid', name='output')
])

model_mnv2.summary()

for images, labels in train_ds:
    predictions = model_mnv2.predict(images)  # Only pass image data
    #print(predictions[:1])
    for pred, label in zip(predictions, labels):
        print("Prediction:", pred, "Actual Label:", label.numpy())# Print the first prediction
    break

Inception.v3: https://towardsdatascience.com/understanding-the-amazon-rainforest-with-multi-label-classification-vgg-19-inceptionv3-5084544fb655

In [90]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D, Flatten
from keras.models import Model

def create_inception_v3_model():
    inceptionv3 = InceptionV3(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    for layer in inceptionv3.layers:
        layer.trainable = False

    # Adding custom layers
    x = inceptionv3.output
    x = GlobalAveragePooling2D()(x)  # Ensure this reduces all spatial dimensions
    x = Dense(4096, activation="relu")(x)
    x = Dropout(0.1)(x)
    output = Dense(3, activation="sigmoid")(x)  # Adjust the number of output units to match the number of classes

    # Creating the final model
    model = Model(inputs=inceptionv3.input, outputs=output)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    return model

# Instantiate and compile the model
inceptionv3_model = create_inception_v3_model()

# Set up the model checkpoint
#model_checkpoint = ModelCheckpoint('inceptionv3_model.h5', monitor="val_accuracy", verbose=1, save_best_only=True)

# Assuming train_ds, X_test, y_test are properly defined
inceptionv3_model.fit(train_ds,
                      validation_data = val_ds, 
                      epochs=10, 
                      #callbacks=[model_checkpoint])
)           


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 51s 14s/step - accuracy: 0.2822 - loss: 17.9969 - val_accuracy: 0.1333 - val_loss: 31.6597
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 36s 13s/step - accuracy: 0.2422 - loss: 29.7807 - val_accuracy: 0.1400 - val_loss: 28.3067
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 38s 14s/step - accuracy: 0.2949 - loss: 25.2096 - val_accuracy: 0.1200 - val_loss: 24.1282
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 36s 13s/step - accuracy: 0.0790 - loss: 24.3685 - val_accuracy: 0.0867 - val_loss: 24.4417
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 36s 13s/step - accuracy: 0.2980 - loss: 23.5756 - val_accuracy: 0.5000 - val_loss: 17.1135
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 37s 13s/step - accuracy: 0.3018 - loss: 19.6772 - val_accuracy: 0.0933 - val_loss: 26.7201
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 38s 14s/step - accuracy: 0.1291 - loss: 27.7690 - val_accuracy: 0.5533 - val_loss: 27.0175
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 36s 13s/step - accuracy: 0.4087 - loss: 31.2693 - val_accuracy: 0.0467 - va

In [92]:
#predict
for images, labels in test_ds:
    predictions = inceptionv3_model.predict(images)  # Only pass image data
    #print(predictions[:1])
    for pred, label in zip(predictions, labels):
        print("Prediction:", pred, "Actual Label:", label.numpy())# Print the first prediction
    break

5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step
Prediction: [1. 1. 1. 1. 1. 1. 1.] Actual Label: [1 0 0 0 0 1 0]
Prediction: [1. 1. 1. 1. 1. 1. 1.] Actual Label: [1 0 0 0 1 1 0]
Prediction: [1. 1. 1. 1. 1. 1. 1.] Actual Label: [0 0 1 0 0 1 0]
Prediction: [1. 1. 1. 1. 1. 1. 1.] Actual Label: [0 0 1 0 0 1 1]
Prediction: [1. 1. 1. 1. 1. 1. 1.] Actual Label: [1 1 0 0 0 0 1]
Prediction: [1. 1. 1. 1. 1. 1. 1.] Actual Label: [1 0 0 0 0 0 0]
Prediction: [1. 1. 1. 1. 1. 1. 1.] Actual Label: [1 1 0 0 0 1 1]
Prediction: [1. 1. 1. 1. 1. 1. 1.] Actual Label: [0 0 1 0 0 1 0]
Prediction: [1. 1. 1. 1. 1. 1. 1.] Actual Label: [0 0 0 1 0 1 0]
Prediction: [1. 1. 1. 1. 1. 1. 1.] Actual Label: [0 0 0 0 0 1 0]
Prediction: [1. 1. 1. 1. 1. 1. 1.] Actual Label: [0 0 0 0 0 1 0]
Prediction: [1. 1. 1. 1. 1. 1. 1.] Actual Label: [0 0 0 0 1 1 0]
Prediction: [1. 1. 1. 1. 1. 1. 1.] Actual Label: [0 0 0 0 0 0 0]
Prediction: [1. 1. 1. 1. 1. 1. 1.] Actual Label: [1 0 0 0 1 1 0]
Prediction: [1. 1. 1. 1. 1. 1. 1.] Actual Label: [0 0 

VGG16: https://towardsdatascience.com/transfer-learning-with-vgg16-and-keras-50ea161580b4

In [11]:
from tensorflow.keras import layers, models
from keras.callbacks import EarlyStopping

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.trainable = False ## Not trainable weights

base_model.summary()

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(3, activation='sigmoid')


model_vgg = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])


from keras.callbacks import EarlyStopping

model_vgg.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model_vgg.fit(train_ds, validation_data = val_ds, epochs=10, batch_size=32, callbacks=[es])

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 314s 90s/step - accuracy: 0.4557 - loss: 0.9414 - val_accuracy: 0.6000 - val_loss: 0.5944
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 266s 87s/step - accuracy: 0.5542 - loss: 0.5646 - val_accuracy: 0.6000 - val_loss: 0.5363
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 271s 89s/step - accuracy: 0.5527 - loss: 0.4981 - val_accuracy: 0.6000 - val_loss: 0.5106
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 311s 86s/step - accuracy: 0.5605 - loss: 0.4687 - val_accuracy: 0.6000 - val_loss: 0.5219
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 279s 90s/step - accuracy: 0.5645 - loss: 0.4562 - val_accuracy: 0.5933 - val_loss: 0.5207
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 263s 89s/step - accuracy: 0.5477 - loss: 0.4356 - val_accuracy: 0.6000 - val_loss: 0.5031


In [12]:
for images, labels in test_ds:
    predictions = model_vgg.predict(images)  # Only pass image data
    #print(predictions[:1])
    for pred, label in zip(predictions, labels):
        print("Prediction:", pred, "Actual Label:", label.numpy())# Print the first prediction
    break

5/5 ━━━━━━━━━━━━━━━━━━━━ 31s 6s/step
Prediction: [0.03325642 0.819588   0.44481274] Actual Label: [0 1 0]
Prediction: [0.03298372 0.7756917  0.28415653] Actual Label: [0 1 0]
Prediction: [0.01971993 0.82212216 0.27173173] Actual Label: [0 1 0]
Prediction: [0.02881509 0.7853935  0.32783452] Actual Label: [0 1 1]
Prediction: [0.02643297 0.80478036 0.36957818] Actual Label: [1 0 1]
Prediction: [0.03034144 0.8443955  0.43575382] Actual Label: [0 0 0]
Prediction: [0.03646141 0.8653222  0.5767437 ] Actual Label: [1 1 1]
Prediction: [0.04153549 0.79053783 0.37910905] Actual Label: [0 1 0]
Prediction: [0.02478776 0.8481702  0.40732867] Actual Label: [0 1 0]
Prediction: [0.03914318 0.8362499  0.39225364] Actual Label: [0 1 0]
Prediction: [0.02710096 0.8233099  0.36825547] Actual Label: [0 1 0]
Prediction: [0.03341808 0.82309467 0.36281875] Actual Label: [0 1 0]
Prediction: [0.02505509 0.7783076  0.33538875] Actual Label: [0 0 0]
Prediction: [0.03583912 0.82933664 0.40166944] Actual Label: [0 1 

ResNet50: https://datagen.tech/guides/computer-vision/resnet-50/

In [107]:
import matplotlib.pyplot as plotter_lib

import numpy as np

import PIL as image_lib

import tensorflow as tflow

from tensorflow.keras.layers import Flatten

from keras.layers import Dense

from tensorflow.keras.models import Sequential

from tensorflow.keras.optimizers import Adam

resnet_model = Sequential()

rn50_base = tflow.keras.applications.ResNet50(
    weights = "imagenet",
    input_shape=(224,224,3)
    )

for each_layer in rn50_base.layers:

        each_layer.trainable=False

resnet_model.add(rn50_base)

resnet_model.add(Flatten())

resnet_model.add(Dense(512, activation='relu'))

resnet_model.add(Dense(7, activation='sigmoid'))

resnet_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

resnet_model.fit(train_ds, validation_data = val_ds, epochs=10)



Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 58s 19s/step - accuracy: 0.1033 - loss: 4.1370 - val_accuracy: 0.0867 - val_loss: 4.4002
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 55s 19s/step - accuracy: 0.1352 - loss: 4.1698 - val_accuracy: 0.0867 - val_loss: 4.3835
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 65s 22s/step - accuracy: 0.1479 - loss: 4.0888 - val_accuracy: 0.0867 - val_loss: 4.3678
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 61s 21s/step - accuracy: 0.1323 - loss: 4.0540 - val_accuracy: 0.0867 - val_loss: 4.3515
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 58s 20s/step - accuracy: 0.1338 - loss: 4.1141 - val_accuracy: 0.0867 - val_loss: 4.3346
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 59s 21s/step - accuracy: 0.1333 - loss: 4.0541 - val_accuracy: 0.0867 - val_loss: 4.3170
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 59s 21s/step - accuracy: 0.1299 - loss: 4.1131 - val_accuracy: 0.0867 - val_loss: 4.2990
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 60s 22s/step - accuracy: 0.1362 - loss: 4.0270 - val_accuracy: 0.0867 - val_loss: 4.2801


In [108]:
for images, labels in train_ds:
    predictions = model_vgg.predict(images)  # Only pass image data
    #print(predictions[:1])
    for pred, label in zip(predictions, labels):
        print("Prediction:", pred, "Actual Label:", label.numpy())# Print the first prediction
    break

8/8 ━━━━━━━━━━━━━━━━━━━━ 51s 6s/step
Prediction: [0.808961   0.3408545  0.12461656 0.16093753 0.05578974 0.320024
 0.44170302] Actual Label: [1 0 0 0 0 1 0]
Prediction: [0.745914   0.3542185  0.1230882  0.18771501 0.05531713 0.31288648
 0.424262  ] Actual Label: [0 0 0 0 0 0 0]
Prediction: [0.74697053 0.34362268 0.10176412 0.16764787 0.04852271 0.28684488
 0.39359173] Actual Label: [1 1 0 0 0 0 1]
Prediction: [0.8429002  0.32981536 0.1052464  0.13855554 0.04150832 0.31023103
 0.44934812] Actual Label: [1 0 0 0 1 1 0]
Prediction: [0.8301681  0.3485725  0.1101438  0.14774078 0.04754231 0.32860145
 0.44287384] Actual Label: [1 1 0 1 0 1 1]
Prediction: [0.77808714 0.3326846  0.09886213 0.15719482 0.03974034 0.28754458
 0.42077675] Actual Label: [0 0 0 0 0 1 0]
Prediction: [0.7709208  0.33817938 0.09353303 0.15388034 0.03791275 0.29584286
 0.40164486] Actual Label: [0 0 0 0 1 0 0]
Prediction: [0.76769364 0.32618228 0.08814462 0.1537395  0.0339381  0.2706281
 0.4042217 ] Actual Label: [1 0 0